**Corretor automático**

<div>
<img src="../assets/spelling.png" width="500" style=" display: block; margin-left: auto; margin-right: auto;"/>
</div>

Vamos criar um aplicativo para verificar o texto em nosso idioma. Enquanto o Microsoft Word e outros editores de texto suportam a verificação ortográfica em idiomas de alto recurso, como inglês ou espanhol, a maioria dos idiomas de baixo recurso não são suportados.

#### Como fazemos isso?
Há duas abordagens possíveis para criar um sistema de verificação ortográfica. 
1. Armazene uma lista enorme com cada palavra na língua. Quando o usuário digita uma palavra, ela é verificada nessa lista.
2. Armazene os *radicais* ou *palavras-raiz* para cada palavra na língua. Quando o usuário digita uma palavra, verifique se a palavra é uma forma válida de uma das radicais. Isto requer conhecimento da morfologia da língua.

Ao contrário do que possamos supor, os aplicativos mais modernos usam **Abordagem 1**. É mais fácil implementar e corre mais rápido.

***

## **Preparação dos dados**

### Carregar o corpus

<div>
<img src="../assets/corpus.png" width="500" style=" display: block; margin-left: auto; margin-right: auto;"/>
</div>

Para este projeto e futuros projetos, usaremos um corpus de [Uspanteko](https://www.ethnologue.com/language/usp). Uspanteko é um língua da família maia, falada no México por cerca de 5.000 pessoas. É uma língua com morfologia aglutinativa e uma escrita que utiliza o alfabeto latino.

Nosso corpus tem 23 arquivos de texto simples em Uspanteko. Vamos ler todos os arquivos e concatená-los juntos.

<div class="alert alert-block alert-warning">
    Se quiser usar dados de outra língua e já tiver um corpus, sinta-se à vontade para usá-la.
</div>

In [ ]:
from typing import List, Dict, Tuple
import os

corpus = ""


# If you're using your own corpus, change this to the correct directory
corpus_directory = "../../corpora/usp"

# Loop over each file in the corpus so we can read it in
for file_name in os.listdir(corpus_directory):
    
    # We will save one corpus entry, 50, for testing
    if file_name == "50.txt":
        continue
        
    
    #  Make sure we only read text files
    if ".txt" not in file_name:
        continue
        
        
    # Read the current file as a string
    file_path = os.path.join(corpus_directory, file_name)
    
    with open(file_path, 'r') as file:
        file_contents = file.read()
        corpus += (file_contents + "\n")
        
print(corpus[:300])

### Pré-processamento
Vamos pré-processar o nosso texto um pouco para a verificação ortográfica.

In [ ]:
# First, strip accent marks (they aren't usually written in Uspanteko).

from util import strip_accents

corpus = strip_accents(corpus)
strip_accents("ójor taq tziij kita' jaa,")

#### **Exercício 1**
Termine a próxima célula para tornar todo o corpus em letra minúscula.
<details>
  <summary>Mostrar resposta</summary>
      <pre style="background-color: honeydew; padding: 10px; border-radius: 5px;"><code style="background: none;">corpus = corpus.lower()</code></pre>
</details>

In [ ]:
# TODO: Finish me!!


print(corpus[:99])

## **Criar uma Lista de Palavras**
Vamos criar uma lista com todas as palavras que ocorrem em nosso corpus. Ignoraremos sinais de pontuação e assumiremos que uma palavra está rodeada de espaços ou pontuação. Além disso, manteremos uma contagem da frequência de cada palavra para ser usada mais tarde.

### Tokenizar palavras usando uma expressão regular

> **Tokenizar** refere-se ao processo de quebra de um sequência em tokens. Tokens podem ser palavras, caracteres ou morfemas. Neste caso, os tokens serão palavras.

Usaremos uma [expressão regular](./skills/regex.ipynb) que procura por grupos de letras e apóstrofes. Quando passamos a regex no nosso texto, cada grupo que encontra é uma palavra distinta.

Por exemplo, no texto seguinte:

```ójor taq tziij kita' jaa```

A regex vai produzir:

```["ojor", "taq", "tziij", "kita'", "jaa"]```

Em Uspanteko, as palavras são sempre divididas por pontuação ou espaço em branco. Portanto, podemos presumir que cada grupo que contém apenas letras constitui uma palavra.

<div class="alert alert-block alert-warning">
Outras línguas podem precisar de uma regex personalizada para detectar palavras. Por favor consulte a lição sobre expressões regulares para informações. Você pode usar uma ferramenta de teste de regex, como <a href='https://regex101.com'>regex101</a>, para certificar-se de que sua expressão regular funciona como espera.
</div>

In [ ]:
import re

# If your language uses some other character within words (like hyphens) you may need to update this regex appropriately
word_regex = r"[\w|\']+"

def tokenize(text: str) -> List[str]:
    return re.findall(word_regex, text)

words = tokenize(corpus)
print(words[:100])

### Criar um léxico

Um **léxico** se refere a todas as palavras utilizadas no corpus

Para criar um léxico, vamos iterar sobre cada palavra do corpus. Usamos um [set](./skills/sets.ipynb) para criar uma lista com todas as palavras independentes no léxico. 

#### **Exercício 2**
Crie um conjunto de todas as palavras independentes no léxico chamado `léxico`. Em seguida, imprima o número de itens do léxico.

<details>
  <summary>Mostrar resposta</summary>
      <pre style="background-color: honeydew; padding: 10px; border-radius: 5px;"><code style="background: none;">lexicon = set()
for word in words:
    lexicon.add(word)</code></pre>
      <pre style="background-color: honeydew; padding: 10px; border-radius: 5px;"><code style="background: none;">print(len(lexicon))
</code></pre>
</details>

In [ ]:
# TODO: Loop over the corpus and add each word to the lexicon


# TODO: Print the number of elements in the lexicon


# Store the lexicon to permanent storage so we can retrieve it later if needed
%store lexicon

## **Construir um Corretor Automático**
Agora, temos um léxico com todas as palavras do corpus e as suas frequências. Estamos prontos para construir nosso programa de corretor automático. 

### Detectar palavras erradas
Vamos criar uma função que receba uma frase e encontre qualquer palavra errada. Cada palavra é verificada para confira se consta no léxico. Se não, é considerada um erro de escrita.

No momento, sempre que encontramos uma palavra que não está no nosso léxico, esta é marcada como um erro (mesmo que seja uma palavra nova, escrita corretamente). Vamos melhorar isso mais tarde.

#### **Exercício 3**
Termine o código a seguir para criar uma função que encontre palavras marcadas com erradas.

<details>
  <summary>Mostrar resposta</summary>
      <pre style="background-color: honeydew; padding: 10px; border-radius: 5px;"><code style="background: none;">def spellcheck(s: str):
    # Pré-processamento e tokenizar
    # TODO: Tirar acentos
    s = strip_accents(s)<br/>
    # TODO: Torne s minúscula
    s = s.lower()<br/>
    # TODO: Tokenizar s
    input_tokenized = tokenize(s)<br/>
    # TODO: Para cada palavra no input, verifique se a palavra está no léxico.
    # Se não, adicione a palavra em "palavras_erradas".
    mispelled_words = []<br/>
    for word in input_tokenized:
        if word not in lexicon:
            mispelled_words.append(word)<br/>
    return mispelled_words
</code></pre>
</details>

<details>
  <summary>Mostrar resposta alternativa</summary>
      <pre style="background-color: honeydew; padding: 10px; border-radius: 5px;"><code style="background: none;">def spellcheck(s: str):
        # Preprocess and tokenize
    # TODO: Strip accents
    s = strip_accents(s)<br/>
    # TODO: Make s lowercase
    s = s.lower()<br/>
    # TODO: Tokenize s
    input_tokenized = tokenize(s)<br/>
    # TODO: Para cada palavra no input, verifique se a palavra está no léxico.
    # Se não, adicione a palavra em "palavras_erradas".
    mispelled_words = [word for word in input_tokenized if word not in lexicon]
    return mispelled_words</code></pre>
</details>

In [ ]:
def spellcheck(s: str):
    # Preprocess and tokenize
    # TODO: Strip accents
    
    
    # TODO: Make s lowercase
    
    
    # TODO: Tokenize s
    
    
    # TODO: For each word in the input, check if the word is in the lexicon.
    # If not, add the word to "mispelled_words".
    
    # TODO: Return mispelled_words

# This sentence has one mispelling ('tzijj')
test_sentence = "Kwand xink'uli'k', re ójr taq tzijj in ák'el na."
mispelled_words = spellcheck(test_sentence)
print("Mispelled words:", mispelled_words)
print("✅ Correct" if mispelled_words == ['tzijj'] else "❌ Incorrect")

### Encontrar as posições de palavra erradas
No futuro, talvez desejemos saber *onde* as palavras erradas ocorrem no texto. A função a seguir encontra a localização de palavras erradas usando a nossa função `corretor automático`.

In [ ]:
def find_mispelled(text: str):
    """Finds mispelled words in a string.
    :return: A list of tuples. Each tuple is (word, index) where `word` is the mispelled word and `index` is the index where it occurs.
    """
    mispelled_words = spellcheck(text)
    
    mispelled_words_and_positions = []
    
    for word in mispelled_words:
        # This regex searches for the given word, surrounded by whitespace or punctuation
        word_regex = f"(^|\W)({word})($|\W)"
        
        # There might be multiple matches if we mispelled a word multiple times
        for match in re.finditer(word_regex, text):
            mispelled_words_and_positions.append({
                'word': word,
                'start': match.start(2),
                'end': match.start(2) + len(word),
                'entity': 'MISPELLED'
            })
        
    # 4. Return the mispelled words, sorted by their position
    return {
        'text': text,
        'entities': sorted(mispelled_words_and_positions, key = lambda x: x['start'])
    }

print(find_mispelled(test_sentence))

## **Tornar o Corretor Automático um Aplicativo Autônomo**

Nossa função `find_mispelled` funciona para detectar erros ortográficos. Mas não é uma ferramenta fácil para um usuário. Vamos criar um aplicativo autônomo que os usuários podem usar.

Vamos usar o [Gradio](https://gradio.app), uma estrutura gratuita que permite transformar código Python em aplicativos compartilháveis. Usar Gradio requer somente três linhas de código.

In [ ]:
import gradio as gr

gr.close_all()

spellchecker = gr.Interface(fn=find_mispelled, inputs="text", outputs="text", live=True)
spellchecker.launch(share=True)

### Melhorar a interface do usuário

Essa estrutura funciona muito bem, e podemos até compartilhar o aplicativo usando o link da web acima. Agora vamos deixar a interface do usuário um pouco mais agradável.

In [ ]:
gr.close_all()

with gr.Blocks(theme=gr.themes.Soft(), title="Uspanteko Spellchecker") as spellchecker:
    gr.Markdown("# Uspanteko Spellchecker")
    
    with gr.Row():
        input_textbox = gr.Textbox(label="Text", info="Text to spellcheck", lines=3)
        output_textbox = gr.HighlightedText(label="Spellchecked", combine_adjacent=True)
    
    gr.Examples(
        examples=["Kwand xink'uli'k', re ójr taq tzijj in ák'el na."],
        inputs=input_textbox,
        outputs=output_textbox,
        fn=find_mispelled,
    )
    
    # Connect the input to the output using our function
    input_textbox.change(find_mispelled, input_textbox, output_textbox)
    
    
spellchecker.launch(share=True)

## **Permitir Palavras Novas**
Existem alguns problemas com o nosso aplicativo. Primeiro, se o usuário digitar uma palavra escrita corretamente, mas que ainda não consta no corpus, será marcada como errada.

Lembra quando salvamos um arquivo de nosso corpus para testes? Vamos ver quantas palavras ainda não vistas ocorrem nesse arquivo.

In [ ]:
test_text = ""

with open("../../corpora/usp/50.txt", 'r') as file:
    test_text = file.read()

len(find_mispelled(test_text)['entities'])

### Implementar o botão "Adicionar palavra" na interface do usuário

Detectamos muitas palavras erradas! Como nosso sistema foi construído utilizando um corpus pequeno, ele não contém todas as palavras válidas na língua. Boas ferramentas de processamento de palavras corrigem esse problema, permitindo que o usuário adicione facilmente palavras novas ao léxico. Então, vamos modificar nossa ferramenta para fazer isso. 

Para isso, introduzimos a ideia de [Gradio State](https://gradio.app/docs/#state). Manter um `Estado` nos permite usar uma variável global em nossas interfaces Gradio. Usaremos uma variável `Estado` para manter o controle das palavras erradas.

#### **Exercício 4**
Termine a função `adicionar_word_to_lexicon` para adicionar a `palavra` ao `léxico`. O argumento `mispelled` é o resultado de chamar `find_mispelled`.

<details>
  <summary>Mostrar resposta</summary>
      <pre style="background-color: honeydew; padding: 10px; border-radius: 5px;"><code style="background: none;">def add_word_to_lexicon(mispelled):
    first_mispelled_word = mispelled['entities'][0]['word']<br/>
    lexicon.add(first_mispelled_word)</code></pre>
</details>

In [ ]:
gr.close_all()

def add_word_to_lexicon(mispelled):
    # TODO: Add the first mispelled word to the lexicon
    

# NEW: We'll use this function when the input changes instead of `find_mispelled`. 
# We need to update 1) the output text, 2) the button, and 3) the global state
def input_text_changed(text):
    mispelled = find_mispelled(text)
    
    should_show_button = len(mispelled['entities']) > 0
    first_mispelled_word = mispelled['entities'][0]['word'] if should_show_button else ''

    # Here we use an "update" function to change the textbox properties. Learn more: https://gradio.app/docs/#update
    add_to_lexicon_button_updater = gr.Button.update(visible=should_show_button, 
                                                     value=f"Add '{first_mispelled_word}' to lexicon")

    return mispelled, add_to_lexicon_button_updater, mispelled

    
# Interface
with gr.Blocks(theme=gr.themes.Soft(), title="Uspanteko Spellchecker") as spellchecker:
    # NEW: A 'State' variable that keeps track of the mispelled words
    mispelled_words_state = gr.State(None)
    
    gr.Markdown("# Uspanteko Spellchecker")
    
    with gr.Row():
        input_textbox = gr.Textbox(label="Text", info="Text to spellcheck", lines=3)
        
        with gr.Column():
            output_textbox = gr.HighlightedText(label="Spellchecked", combine_adjacent=True)
            
            # NEW: A button that adds the first mispelled word to the lexicon
            add_word_button = gr.Button(value=f"Add word to lexicon", visible=False)
            
    gr.Examples(
        examples=["Kwand xink'uli'k', re ójr taq tzijj in ák'el na."],
        inputs=input_textbox,
        outputs=output_textbox,
        fn=find_mispelled,
    )
    
    input_textbox.change(input_text_changed, input_textbox, [output_textbox, add_word_button, mispelled_words_state])
    
    # NEW: Run a function when the button is called, then update the state
    add_word_button \
        .click(add_word_to_lexicon, [mispelled_words_state]) \
        .then(input_text_changed, input_textbox, [output_textbox, add_word_button, mispelled_words_state])
    
        
spellchecker.launch(share=True)

Agora, podemos facilmente adicionar qualquer palavra escrita corretamente ao nosso léxico, e ela não será marcada como erro no futuro!

## **Correção ortográfica**
Por último, seria bom atualizar o nosso corretor ortográfico para que dê sugestões de ortografia correta quando encontra um erro. Para isso, precisamos determinar qual palavra em nosso léxico é mais próxima do que foi digitado. Usaremos **editar distância**, que mede quantas edições (adições, exclusões, alterações) são necessárias para transformar uma sequência em outra.

In [ ]:
import nltk

def spelling_suggestions(mispelled_word, n):
    # 1. Calculate the edit distance between the word and every word in the lexicon
    candidate_spellings = []
    
    for word in lexicon:
        edit_distance = nltk.edit_distance(word, mispelled_word)
        candidate_spellings.append((word, edit_distance))
    
    # 2. Find the top n closest words
    sorted_candidates = sorted(candidate_spellings, key=lambda x: (x[1]))
    top_n_candidates = sorted_candidates[:n]
    top_n_words_only = [candidate[0] for candidate in top_n_candidates]
    return top_n_words_only

spelling_suggestions("tzijj", 3)

### Implementar sugestões na interface do usuário
Vamos implementar esta funcionalidade na interface do usuário!

In [ ]:
gr.close_all()

def input_text_changed2(text):
    mispelled = find_mispelled(text)
    
    should_show_button = len(mispelled['entities']) > 0
    first_mispelled_word = mispelled['entities'][0]['word'] if should_show_button else ''

    add_to_lexicon_button_updater = gr.Button.update(visible=should_show_button, 
                                                     value=f"Add '{first_mispelled_word}' to lexicon")
    
    # NEW: Generate suggestions for the closest spelling
    suggestions = spelling_suggestions(first_mispelled_word, 3)
    suggestions_text = " | ".join(suggestions)

    return mispelled, add_to_lexicon_button_updater, mispelled, suggestions_text

    
# Interface
with gr.Blocks(theme=gr.themes.Soft(), title="Uspanteko Spellchecker") as spellchecker:
    mispelled_words_state = gr.State(None)
    
    gr.Markdown("# Uspanteko Spellchecker")
    
    with gr.Row():
        input_textbox = gr.Textbox(label="Text", info="Text to spellcheck", lines=3)
        
        with gr.Column():
            output_textbox = gr.HighlightedText(label="Spellchecked", combine_adjacent=True)
            add_word_button = gr.Button(value=f"Add word to lexicon", visible=False)
            
            # NEW: Show suggested spellings
            suggestions_textbox = gr.Textbox(label="Suggestions", interactive=False)
            
    gr.Examples(
        examples=["Kwand xink'uli'k', re ójr taq tzijj in ák'el na."],
        inputs=input_textbox,
        outputs=output_textbox,
        fn=find_mispelled,
    )
    
    input_textbox.change(input_text_changed2, input_textbox, [output_textbox, add_word_button, mispelled_words_state, suggestions_textbox])
    
    add_word_button \
        .click(add_word_to_lexicon, [mispelled_words_state]) \
        .then(input_text_changed2, input_textbox, [output_textbox, add_word_button, mispelled_words_state, suggestions_textbox])
    
        
spellchecker.launch(share=True)

## **Resumo**
Neste tutorial, construímos uma ferramenta de verificação ortográfica para uma linguagem de baixo recurso. Isso incluiu:
- A construção do léxico a partir dos textos originais
- A detecção de palavras erradas
- A previsão da ortografia correta usando métricas de similaridade

### **Desafios**
1. No momento, nosso verificador ortográfico só permite adicionar a primeira palavra errada ao léxico. Melhore esta funcionalidade criando dois botões novos. Um botão permitirá que o usuário **Adicione todas** as palavras erradas ao léxico. O outro botão permite **ignorar** uma palavra errada e passar para a próxima. 
2. Nosso verificador ortográfico exibe sugestões, mas não permite utilizá-las. Substitua a caixa de texto por botões para cada sugestão. Clicando em uma sugestão, ela deve substituir a palavra errada no campo de texto.